In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pycountry
import geoplot as gplt
import geopandas as gpd
import geoplot.crs as gcrs
import imageio
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import mapclassify as mc
import os.path
from pickling import pickle_load
from helpers import *
print("import completed")

#data folder path
data_folder = './Data/'
path_ba = data_folder + 'BeerAdvocate/'
path_rb = data_folder + 'RateBeer/'
path_md = data_folder + 'matched_beer_data/'




 # Pipeline
 What was written on the sheet of paper:
 Data sources:
 - ratebeer, beer advocate, matched ds
 - how do they compare?
 - enrichement
 Data model:
 - table (maybe network)
 Data Wrangling:
 - column description
 - should we combine or merge rb and ba? On what can we merge? Do both ds have the same set of beer-styles?
 - Sanity check: do number of reviews in users table match number of ratings or number of reviews?
 - data cleaning (null values, duplicates, ...)
 - completeness
 - data types
 - language
 - country-code (England-uk?)
 - matching avg
 - crawl beer styles from https://www.beeradvocate.com/beer/styles/ and https://www.ratebeer.com/beerstyles/ (for some textual description, we do have style columns in beer tables)
 Andreaas Idea:
 - vocabulary: <-> Location

 ## Beer Advocate

 ### Beers

 Source of column description: https://www.beeradvocate.com/community/threads/beeradvocate-ratings-explained.184726/
<

 - beer_id
 - name
 - brewery_id
 - brewery_name
 - style
 - nbr_ratings
 - nbr_reviews
 - avg
 - ba_score (The BA Score is the beer's overall score based on its ranking within its style category. It's based on the beer's truncated (trimmed) mean and a custom Bayesian (weighted rank) formula that takes the beer's style into consideration. Its purpose is to provide consumers with a quick reference using a format that's familiar to the wine and liquor worlds.)
       95-100 = world-class
       90-94 = outstanding
       85-89 = very good
       80-84 = good
       70-79 = okay
       60-69 = poor
       < 60 = awful
 - bros_score
 - abv (alcohol by volume)
 - avg_computed
 - zscore
 - nbr_matched_valid_ratings
 - avg_matched_valid_ratings

In [ ]:
#read in beers.csv
df_ba_beers = pd.read_csv(path_ba + 'beers.csv')
df_ba_beers.set_index(df_ba_beers['beer_id'], inplace = True)
df_ba_beers.pop('beer_id')
df_ba_beers.sort_index(inplace=True)
df_ba_beers.head()

In [ ]:
# Check for null values
print(df_ba_beers.isna().sum())
print(f"number of beers in the dataframe: {len(df_ba_beers)}")
# We do have a few nan values in avg, ba_score, bros_score, abv, avg_comuted,zscore and avg_matched_valid_ratings
# According to https://www.beeradvocate.com/community/threads/beeradvocate-ratings-explained.184726/ it may be, that averages are not computed for beers with less than 10 reviews/ratings (not sure what the difference is)

In [ ]:
# Are the avg nan values unrated beers?
df_ba_beers[df_ba_beers["avg"].isna() & df_ba_beers["nbr_ratings"]!=0 ].head()
# Yes, they are. O/w this dataframe would not be empty.
# Do these beers have reviews?
df_ba_beers[df_ba_beers["avg"].isna() & df_ba_beers["nbr_reviews"]!=0 ].head()
# Neither.


In [ ]:
# What is the column "zscore" the zscore of? avg or ba_score ? 
#avgscore_mean = df_ba_beers["avg_computed"].mean()
#avgscore_var = df_ba_beers["avg_computed"].var()
#df_ba_beers["zscore_avg"] = df_ba_beers["avg"].apply(lambda x: (x-avgscore_mean)/avgscore_var)
#df_ba_beers[~df_ba_beers['zscore'].isnull()]

In [ ]:
# Give basic statistics about the ba beer table
df_ba_beers.describe()
# TODO plot some interesting values like ratings vs review distribution...


 ###### data visualizations
 Histogram of alcohol content

 Number of beers per beer style
ax.set_yticks(style_counts.index(), labels=people)

In [ ]:
# american and german styles:
american_style = [x for x in style_counts.index if "American" in x]
german_style = [x for x in style_counts.index if ("German" in x or "Munich" in x)]
print("Number of american beerstyles in BA dataset",len(american_style))
print(american_style)
print("Number of german beerstyles in BA dataset",len(german_style))
print(german_style)


 Alcohol content among some of the most popular beer styles

 Let's create a beer ranking
 information about "Bud Light" - according to Matthieu most popular us beer (double check))

 ### Breweries
 - brewery_id -> id
 - location
 - name
 - nbr_beers

 #### Breweries location processing
 TODO: document what's going on...

In [ ]:
#removing html tags
df_ba_breweries['location'] = df_ba_breweries['location'].apply(lambda x : x.split('<', 1)[0])
#separating country from states (for USA)
df_ba_breweries['country'] = df_ba_breweries['location'].apply(lambda x : x.split(',', 1)[0])
country_list = df_ba_breweries['country'].unique()
#the function find_iso do a matching of common "non-official" country name to their iso code using the pycountry library
#For example, "united states" is matched to "United states of America" and is assigned the iso code USA
country_iso = {x: find_iso(x) for x in country_list}
df_ba_breweries['country_code'] = df_ba_breweries['country'].apply(lambda x: country_iso[x])
print(df_ba_breweries.head())



In [ ]:
print("number of missing country codes:",df_ba_breweries['country_code'].isna().sum())
print(df_ba_breweries.isna().sum())
# TODO only country codes are unknown for a few breweries...


 ##### Create a us-specific dataframe
special detailed dataframe for usa

In [ ]:
print("number of missing country codes:",df_ba_breweries_usa['country_code'].isna().sum())
print("number of unknown states",df_ba_breweries_usa['states'].isna().sum())
print(df_ba_breweries_usa.isna().sum())
# TODO Some states are unknown


 Beer count per country
beer count per country

 ##### data visualizations
 Geographical distribution of beers
number of beers per country (log)

 Number of beers per state (USA)

 ### Users
 - nbr_ratings
 - nbr_reviews
 - user_id
 - user_name
 - joined
 - location
 We do not know 1 username, 2652 join dates and 31279 locations (i.e. they are nan)
location processing
removing html tags
separating country from states (for USA)
 54 missing country codes...

In [ ]:
df_ba_users_usa = df_ba_users[df_ba_users['country'] == 'United States']
states = df_ba_users_usa['location'].apply(lambda x : str(x.split(',', 1)[1])[1:] if ',' in x else None)
df_ba_users_usa = df_ba_users_usa.assign(states = states)
print(df_ba_users_usa.sample(10))


In [ ]:
users_states = df_ba_users_usa.groupby(df_ba_users_usa["states"]).sum().sort_values(by = 'nbr_ratings', ascending = False)
users_contiguous_usa = gpd.read_file(gplt.datasets.get_path("contiguous_usa"))
users_contiguous_usa = users_contiguous_usa.merge(users_states, how = "left", left_on="state", right_index = True)
print(users_contiguous_usa.sample(10))
users_contiguous_usa = users_contiguous_usa.sort_values(by = "nbr_ratings", ascending = False)
#TODO: not done yet...
users_contiguous_usa["ratings_per_capita"] = users_contiguous_usa.apply(lambda x: 0 if x["nbr_ratings"] == np.nan else x["nbr_ratings"]/x["population"], axis = 1)
gplt.choropleth(users_contiguous_usa, hue="ratings_per_capita", legend=True)
plt.title("nb of ratings per capita per state")
plt.show()


 Number of reviews per user
 ba_users_review_counts = df_ba_users['nbr_reviews'].value_counts()
 logbins = np.geomspace(ba_users_review_counts.min(), ba_users_review_counts.max(), 100)
 print(logbins)
 plt.hist(ba_users_review_counts,  density=True, log=True,bins=logbins)
 plt.xscale("log")
 plt.xlabel("Number of reviews per user")
 plt.ylabel("Frequency")
 plt.title("Distribution of number of review per users")
 plt.show()
 TODO: something seems wrong about this plot. Are there no users with less than 100 reviews?
 TODO: plot the same for ratings
 TODO: what is the difference between ratings and reviews?

 Sanity Check
 Number of users with 4 reviews/ratings
 Are users unique?
 We lose 4 users when grouping by user_id

 ### Reviews
 The difference in columns between reviews and ratings is only the missing boolean
 - beer_name
 - beer_id
 - brewery_name
 - brewery_id
 - style
 - abv
 - date
 - user_name
 - user_id
 - appearance
 - aroma
 - palate
 - taste
 - overall
 - rating
 - text
 For performance reasons, we will not always load the reviews, they are a subset of ratings.
 df_ba_reviews = pickle_load(path_ba + "reviews.txt")
 df_ba_reviews.set_index(df_ba_reviews["date"], inplace = True)
 df_ba_reviews.head()

In [ ]:
# df_ba_reviews.describe()
# df_ba_reviews.isna().sum()


 #### Data visualizations
 BA reviews as time-series
 ba_reviews_month = df_ba_reviews.groupby(df_ba_reviews.index.to_period('M')).size()
 ba_reviews_month.loc[ba_reviews_month.index.year > 2010].plot()

 ### Ratings are a superset of reviews, they contain a lot more nan values both in numerical score and in text review
 - beer_name
 - beer_id
 - brewery_name
 - brewery_id
 - style
 - abv
 - date
 - user_name
 - user_id
 - appearance
 - aroma
 - palate
 - taste
 - overall
 - rating
 - text
 - review
 df_ba_ratings = pickle_load(path_ba + "ratings.txt",load_with_text=True)
 df_ba_ratings.set_index(df_ba_ratings["date"], inplace = True) # Set date as index for ratings
 df_ba_ratings = df_ba_ratings.convert_dtypes(infer_objects=True) # infer correct string types (does not filter nan values)
 df_ba_ratings[["overall","taste","palate","aroma","appearance","abv"]] = df_ba_ratings[["overall","taste","palate","aroma","appearance","abv"]].apply(lambda x : pd.to_numeric(x,errors="coerce")) # coerce to numeric
 df_ba_ratings = df_ba_ratings.astype({"review":bool}) # convert to boolean
 df_ba_ratings["text"] = df_ba_ratings["text"].apply(lambda x: x if str.lower(x) != "nan" else "") # convert nan to empty string
 df_ba_ratings = df_ba_ratings.dropna() # drop nan values
 print(df_ba_ratings.info()) # print info
 print("Number of nan values",df_ba_ratings.isna().sum())
 print("Number of non-reviews:",df_ba_ratings[df_ba_ratings["review"] == False].shape[0])
 print("Number of text that is 'nan':",df_ba_ratings[df_ba_ratings["text"].str.lower() == "nan"].shape[0])
 for column in ["beer_name","beer_id","brewery_name","brewery_name","style","user_name","user_id"]:
         print(f"number of values in {column} that is 'nan'", df_ba_ratings[df_ba_ratings[column].str.lower() == "nan"].shape[0])
 TODO: convert to style to categorical, it might make sense to also change datatype of beer_id and brewery_id to numeric

 #### Data visualizations
 BA ratings as time-series
 ba_ratings_month = df_ba_ratings.groupby(df_ba_ratings.index.to_period('M')).size()
 ba_ratings_month.plot()

 #### Ratings vs Reviews
 Compute intersection
 df_ba_ratings_reviews = df_ba_ratings.merge(df_ba_ratings, how = "inner", on = ["beer_name", "beer_id", "brewery_name", "brewery_id", "style", "abv",  "user_name", "user_id", "appearance", "aroma", "palate", "taste", "overall", "rating"])

In [ ]:
# df_ba_ratings_reviews.describe()
# df_ba_ratings_reviews.isna().sum()

In [ ]:
# print("size of intersection in review/ratings BA", df_ba_ratings_reviews[~df_ba_ratings_reviews.isnull().any(axis=1)].count())
# print("size of reviews BA", df_ba_ratings[~df_ba_ratings.isnull().any(axis=1)].count())
# Reviews are a subset of ratings for the BA dataset!


In [ ]:
# df_ba_ratings[ df_ba_ratings["text"].isna()].count()


In [ ]:
# df_ba_ratings_gb_text = df_ba_ratings.groupby("text")


In [ ]:
# df_ba_ratings_gb_text.describe()


 ## Rate Beer

 ### Beers
 - beer_id
 - beer_name
 - brewery_name
 - style
 - nbr_ratings
 - overall_score (A score that ranks this beer against all other beers on RateBeer (max=100))
 - style_score (A score that ranks this beer against all beers within its own style (max=100))
 - avg (weighted average)
 - abv (alcohol content)
 - avg_computed (average rating)
 - zscore
 - nbr_matched_valid_ratings
 - avg_matched_valid_ratings

In [ ]:
#read in beers.csv
df_rb_beers = pd.read_csv(path_rb + 'beers.csv')
df_rb_beers.head()

# print("Total # of beers: ", df_rb_beers[df_rb_beers.columns[0]].count())


In [ ]:
#check if # of elements missing

for column in df_rb_beers:
    print(column, " : ", df_rb_beers[column].isnull().sum())


In [ ]:
# check if any beers appear multiple times
# we should not sum them up... because avg depends non-linearly on number of ratings

df_rb_beers_same_name = df_rb_beers[df_rb_beers["beer_name"].duplicated(keep=False)]
df_rb_beers_same_name.sort_values(by=["beer_name"])
print("# of beers with identical name: ", df_rb_beers_same_name[df_rb_beers_same_name.columns[0]].count())

# but we have beers with same name from different breweries
# only make list with same name and same brewery

df_rb_beers_same_name_and_brewery = df_rb_beers_same_name[df_rb_beers_same_name["brewery_name"].duplicated(keep=False)]
df_rb_beers_same_name_and_brewery.sort_values(by="beer_name")
print("# of beers with identical name & brewery: ", df_rb_beers_same_name_and_brewery[df_rb_beers_same_name_and_brewery.columns[0]].count())
# how important are those beers
print("# of ratings for beers with identical name & brewery: ", df_rb_beers_same_name_and_brewery["nbr_ratings"].sum())
print("gives an average of {} ratings/beer.".format(df_rb_beers_same_name_and_brewery["nbr_ratings"].sum()/df_rb_beers_same_name_and_brewery[df_rb_beers_same_name_and_brewery.columns[0]].count()))


In [ ]:
# list all beer styles

rb_styles = df_rb_beers["style"].value_counts()
print("There exist {} different beer styles".format(len(rb_styles)))

fig, ax = plt.subplots()
ax.barh(rb_styles.index, rb_styles, align='center')
ax.invert_yaxis()  # labels read top-to-bottom
ax.tick_params(axis='y', which='major', labelsize=4)
ax.set_xlabel('numbers of beer')
ax.set_ylabel("Beer style")
ax.set_title('number of beer per style')
plt.show()


In [ ]:
# check if there is any difference etween avg_computed and avg_matched valid_ratings

no_match = ~df_rb_beers["avg_matched_valid_ratings"].isnull() & df_rb_beers["avg_matched_valid_ratings"].ne(df_rb_beers["avg_computed"])
print("there are {} dataframes where avg_matched_valid_ratings is not NaN or not equal avg_computed".format(df_rb_beers[no_match].shape[0]))


In [ ]:
# how many ratings did the beers get?

fig, ax = plt.subplots()
ax.hist(df_rb_beers["nbr_ratings"], bins=1000, log=True)
ax.set_xlabel("Number of ratings")
ax.set_ylabel("Number of beers")
plt.show()

df_rb_beers.sort_values(by=["nbr_ratings"], ascending=False).head()


In [ ]:
# how many overall ratings per style
rb_ratings = df_rb_beers.groupby(["style"])["nbr_ratings"].sum()
# Average Rating per beer per beer style

rb_avg_rating_per_style = rb_ratings/rb_styles.sort_index()

fig, ax = plt.subplots()
ax.barh(rb_avg_rating_per_style.index, rb_avg_rating_per_style, align='center')
# ax.invert_yaxis()  # labels read top-to-bottom
ax.tick_params(axis='y', which='major', labelsize=4)
ax.set_xlabel('Average ratings per beer per style')
ax.set_ylabel("Beer style")
ax.set_title('Average ratings per beer per style')
plt.show()




 ### Breweries
 split US states

In [ ]:
# add country code
country_list = df_rb_brew['country'].unique()
country_iso = {x: find_iso(x) for x in country_list} #look up table
df_rb_brew['country_code'] = df_rb_brew['country'].apply(lambda x: country_iso[x])


In [ ]:

df_rb_brew_per_country = df_rb_brew.groupby(["country_code"])["country_code"].count()
print(df_rb_brew_per_country.head())
df_rb_beers_per_country = df_rb_brew.groupby(["country_code"])["nbr_beers"].sum()
print(df_rb_beers_per_country.head())


In [ ]:
# nb of breweries per country

print(type(df_rb_brew_per_country.sort_values(ascending=False).head()))
# Uncomment me for a world map:

# world = gpd.read_file(data_folder + "maps/world-administrative-boundaries.shp", encoding = 'utf-8')
# world = world.merge(df_rb_brew_per_country, how = "left", left_on="iso3", right_index = True, )
# world = world.sort_values(by = "country_code", ascending = False)
# world['log_beers'] = world['country_code'].apply(lambda x: np.log10(x) if x >= 1 else 0)
# #number of breweries per country (log)
# scheme = mc.FisherJenks(world['log_beers'], k=8)
# gplt.choropleth(world, hue="log_beers", legend=True, scheme=scheme)
# plt.title("nb of beer per country (log10)")
# plt.show()


In [ ]:
# nb of beers per country
print(df_rb_beers_per_country.sort_values(ascending=False).head())
# Uncomment me for a world map:

world = gpd.read_file(data_folder + "maps/world-administrative-boundaries.shp", encoding = 'utf-8')
world = world.merge(df_rb_beers_per_country, how = "left", left_on="iso3", right_index = True, )
world = world.sort_values(by = "nbr_beers", ascending = False)
world['log_beers'] = world['nbr_beers'].apply(lambda x: np.log10(x) if x >= 1 else 0)

#number of beers per country (log)
scheme = mc.FisherJenks(world['log_beers'], k=8)
gplt.choropleth(world, hue="log_beers", legend=True, scheme=scheme)
plt.title("nb of beer per country (log10)")
plt.show()


 ### Users
 remove lines with nan
 split US states
 remove missing country codes

In [ ]:
# Users per country

rb_users_per_country = df_rb_users.groupby(["country"])["country"].count().sort_values(ascending=False)
print(type(rb_users_per_country))


In [ ]:
# Number of ratings per country

rb_ratings_per_country = df_rb_users.groupby(["country"])["nbr_ratings"].sum().sort_values(ascending=False)
print(type(rb_ratings_per_country))


In [ ]:
# average nb of rating per user

rb_avg_rating_per_user_per_country = rb_ratings_per_country/rb_users_per_country

fig, ax = plt.subplots()
ax.hist(rb_avg_rating_per_user_per_country.sort_values(ascending=False).head(80), bins=200)
ax.set_xscale('log')
ax.set_xlabel("Avg ratings per user per country")
ax.set_ylabel("Number of countries")
ax.set_title("Top 80 countries with highest avg ratings per user")
plt.show()










 ### Reviews
 TODO: improve comments... (justification for using reviews instead of ratings for RB)
 df_rb_reviews = pickle_load(path_rb + "reviews.txt",load_with_text=True)
 df_rb_reviews.set_index(df_rb_reviews["date"], inplace = True) # Set date as index for ratings
 df_rb_reviews = df_rb_reviews.convert_dtypes(infer_objects=True) # infer correct string types (does not filter nan values)
 df_rb_reviews[["overall","taste","palate","aroma","appearance","abv","user_id"]] = df_rb_reviews[["overall","taste","palate","aroma","appearance","abv","user_id"]].apply(lambda x : pd.to_numeric(x,errors="coerce")) # coerce to numeric
 df_rb_reviews["text"] = df_rb_reviews["text"].apply(lambda x: x if str.lower(x) != "nan" else "") # convert nan to empty string
 df_rb_reviews = df_rb_reviews.dropna() # drop nan values
 print(df_rb_reviews.info()) # print info
 print("Number of nan values",df_rb_reviews.isna().sum())
 print("Number of text that is 'nan':",df_rb_reviews[df_rb_reviews["text"].str.lower() == "nan"].shape[0])
 for column in ["beer_name","beer_id","brewery_name","brewery_name","style","user_name","user_id"]:
         print(f"number of values in {column} that is 'nan'", df_rb_reviews[df_rb_reviews[column].str.lower() == "nan"].shape[0])

In [ ]:
# print(df_rb_reviews.head())


 ### Ratings
 df_rb_ratings = pickle_load(path_rb + "ratings.txt")
 df_rb_ratings.head()

In [ ]:
# df_rb_ratings.isna().sum()


 #### Ratings vs Reviews
 Compute intersection
 df_ba_ratings_reviews = df_ba_ratings.merge(df_ba_reviews, how = "inner", on = ["beer_name", "beer_id", "brewery_name", "brewery_id", "style", "abv",  "user_name", "user_id", "appearance", "aroma", "palate", "taste", "overall", "rating"])

In [ ]:
# df_ba_ratings_reviews.describe()
# df_ba_ratings_reviews.isna().sum()

In [ ]:
# print("size of intersection in review/ratings BA", df_ba_ratings_reviews[~df_ba_ratings_reviews.isnull().any(axis=1)].count())
# print("size of reviews BA", df_ba_reviews[~df_ba_reviews.isnull().any(axis=1)].count())


 ## Matched Dataset
 column dropping justification:
 - ba_score: only available in ba dataset, not really linear, not available for every review (only for the aggregation of all reviews) and hard to interpret (rather use user avg.)
 - bros_score: wtf ? only available in ba dataset
 - anything related to reviews
 - overall_score: same argument as ba_score
 - avg_matched_...: we don't care about matching the two datasets since we're testing each hypothesis two times

the rest we keep just in case
issues:
 - what is the zscore
 - what is the avg_matched_valid_ratings ?
 - what is diff and sim ?

 ### Beers
loading and naming columns
infos:
print(f"number of beers in BeerAdvocate dataset {len(df_ba_beers)}")
so we lost about 84% of the beers, but still 45k is significative.
is it interesting to compute the avg nb of reviews for beers we lost ? => don't know how to do (A \ AintersectB)
nan checks:
print(df_beers.isna().sum())
not sure: we remove beers that didn't get at least 1 review in each ds ?
no NAN lefts !
do abv contents are the same in both datasets? Yes they are
do names match ? Most of them don't. BA don't include the brewery name in the beer name but RB often does

 ### Breweries
loading and renaming columns
updating breweries list and beer count according to the beer we removed in the previous section
do number of beers per brewery matches between the two dataset ?
no, but we notice that it is ony due to minor difference in the name, so we leave them as it is

 ### Users

 ### Users Approx

 ### Reviews

 ### Ratings

 #### Ratings with text ba
 df_ratings_with_text_ba = pickle_load(path_md + "ratings_with_text_ba.txt")

 #### Ratings with text rb

 # Create Dataframes for Ratings including userdata filtered on matched beer dataset

In [ ]:
# ## BeerAdvocate
# Filter BA Ratings to only keep beers in matched beer dataset
# df_md_beer_ids = df_beers["beer_id_ba"].unique().astype(str)
# df_ba_beer_ids = df_ba_ratings["beer_id"].unique()
# df_ba_ratings_filtered_md_beers = df_ba_ratings[df_ba_ratings["beer_id"].isin(df_md_beer_ids)]
# df_ba_ratings_filtered_beers_merged_users = df_ba_ratings_filtered_md_beers.merge(df_ba_users, left_on="user_id", right_on="user_id")
# print(df_ba_ratings_filtered_beers_merged_users.isnull().sum())
# print(df_ba_ratings_filtered_beers_merged_users.head())
# print(df_ba_ratings_filtered_beers_merged_users.info())
pickle_filename = "df_ba_ratings_filtered_beers_merged_users.pickle"
# df_ba_ratings_filtered_beers_merged_users.to_pickle(f"Data/{pickle_filename}")
df_ba_ratings_filtered_beers_merged_users = pd.read_pickle(f"Data/{pickle_filename}")
print(df_ba_ratings_filtered_beers_merged_users.head())
print(df_ba_ratings_filtered_beers_merged_users.info())
df_ba = df_ba_ratings_filtered_beers_merged_users




In [ ]:
# ## RateBeer
# Filter RB Ratings to only keep beers in matched beer dataset

# df_md_beer_ids = df_beers["beer_id_rb"].unique().astype(str)
# df_rb_beer_ids = df_rb_reviews["beer_id"].unique()
# df_rb_reviews_filtered_md_beers = df_rb_reviews[df_rb_reviews["beer_id"].isin(df_md_beer_ids)]
# df_rb_reviews_filtered_beers_merged_users = df_rb_reviews_filtered_md_beers.merge(df_rb_users, left_on="user_id", right_on="user_id")
# print(df_rb_reviews_filtered_beers_merged_users.isnull().sum())
# print(df_rb_reviews_filtered_beers_merged_users.head())
# print(df_rb_reviews_filtered_beers_merged_users.info())
pickle_filename = "df_rb_reviews_filtered_beers_merged_users.pickle"
# df_rb_reviews_filtered_beers_merged_users.to_pickle(f"Data/{pickle_filename}")
df_rb_reviews_filtered_beers_merged_users = pd.read_pickle(f"Data/{pickle_filename}")
print(df_rb_reviews_filtered_beers_merged_users.head())
print(df_rb_reviews_filtered_beers_merged_users.info())


In [ ]:
# TODO: warning when comparing ratings between RB and BA: Appearance, Aroma, Paalte, Taste and overall have different ranges in both datasets (rating to be double checked)
# for the average rating comparison:
plt.hist(df_beers["avg_computed_ba"], bins=50, alpha=0.5, label='BeerAdvocate rating', density = True)
plt.hist(df_beers["avg_computed_rb"], bins=50, alpha=0.5, label='RateBeer rating', density = True)
plt.title("histogram of ratings from both dataset within the merge dataset")
plt.legend(loc='upper right')
plt.xlabel("rating")
plt.ylabel("density")
plt.show()
#We notice that rate beer is more critical in avg. that beer advocate. Caution should be taken if two datasets are merged

 # without propensity matching:

In [ ]:
df_ba_american = df_ba.loc[df_ba["location_ba"].split()[0:2] == "United States"]
